In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
! pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 38.5 MB/s eta 0:00:00


In [3]:


import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil
from tqdm import tqdm

In [4]:
# initialize Pose estimator
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
pose = mp_pose.Pose(
    model_complexity=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

In [6]:
def stream_pose(model, video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        # read frame from capture object
        _, frame = cap.read()

        try:
            # convert the frame to RGB format
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # process the RGB frame to get the result
            results = model.process(RGB)

            print(results.pose_landmarks)
            # draw detected skeleton on the frame
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # show the final output
            cv2.imshow('Output', frame)
        except Exception  as e:
          print(e)
          break
        if cv2.waitKey(1) == ord('q'):
          break
    cap.release()
    cv2.destroyAllWindows()


def gen_skeleton(frame, connections, height, width,
                 mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
                 ):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles 
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines 
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3


def pose_estimator(model, path):
    # create capture object
    cap = cv2.VideoCapture(path)
    if (cap.isOpened() == False): 
        print("Error reading video file")
    
    # We need to set resolutions.
    # so, convert them from float to integer.
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    size = (frame_width, frame_height)

    skeleton_video = []
    while cap.isOpened():
        # read frame from capture object
        _, frame = cap.read()

        try:
            # convert the frame to RGB format
            RGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # process the RGB frame to get the result
            results = pose.process(RGB)
            try:
                frame_result = results.pose_landmarks.landmark
                point_list = []
                for idx, landmark in enumerate(frame_result):
                    # print(idx, landmark.x)
                    # print(type(landmark))
                    coord = [landmark.x, landmark.y, landmark.z]
                    point_list.append(coord)
                skeleton_video.append(point_list)
            except Exception as inst:
                print(inst)
                pass
            
        except Exception as inst:
            print(inst)
            break
        if cv2.waitKey(1) == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

    return np.array(skeleton_video), frame_height, frame_width

def gen_video(points, save_file,file_id, frame_h, frame_w, 
                 mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)],
              temp_dir="temp",
              ):
    os.makedirs(f"{temp_dir}/{file_id}",exist_ok=True)
    #size = (frame_w, frame_h)
    #result = cv2.VideoWriter(save_file, 
    #                     cv2.VideoWriter_fourcc(*'MJPG'),
    #                     10, size)

    #for frame in [points[-110]]:
    for __id,frame in enumerate(points):
        __id = "0"*(6-len(str(__id))) + str(__id)
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        skel_image = cv2.resize(skel_image,[int(frame_w*0.2),int(frame_h*0.2)])
        plt.imsave(f"{temp_dir}/{file_id}/{__id}.png",skel_image)

    os.system(f"ffmpeg -framerate 10 -pattern_type glob -i '/content/temp/{file_id}/*.png' -c:v libx264 -pix_fmt yuv420p '{save_file}'")
    shutil.rmtree(f"/content/temp/{file_id}")
        #result.write(skel_image[:,:,::-1])
    #result.release()

def save_arrays_file(skeletal_frames,video_size,location):
  print(skeletal_frames.shape)
  np.savez(location,coords=skeletal_frames,video_size=video_size)

In [7]:
#shutil.rmtree("/content/temp")

In [8]:
skel_res, fh, fw = pose_estimator(pose, '/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/video_dataset/sitting_down/a109-1310C.mp4')
gen_video(skel_res, 'sample_poes_gen.mp4',"sample_vid", fh, fw)
save_arrays_file(skel_res,[fh,fw],"bdv.npz")

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [9]:
file_load = np.load("/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/Consolidated/PAMPA2/Skeletons/skeleton_mediapipe.npz")
file_load.files

['arr_0', 'arr_1', 'arr_2']

In [10]:
file_load["arr_0"].shape,file_load["arr_1"].shape,file_load["arr_2"].shape

((177,), (177, 60, 36), (177,))

In [11]:
data_fol = "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/Consolidated/UTD-MHAD/Videos"
path_parts = data_fol.split("/Datasets/")[1]

model_ident = "MediaPose"
set_of_classes = [x for x in os.listdir(data_fol)]

In [12]:
set_of_classes

['Swipe left',
 'Swipe right',
 'Wave',
 'Clap',
 'Throw',
 'Arm cross',
 'Basketball shoot',
 'Draw X',
 'Draw triangle',
 'Bowling',
 'Boxing',
 'Baseball swing',
 'Tennis swing',
 'Arm curl',
 'Tennis serve',
 'Push',
 'Knock',
 'Catch',
 'Pickup and throw',
 'Jog',
 'Walk',
 'Sit to stand',
 'Stand to sit',
 'Lunge',
 'Squat',
 'Draw circle (clockwise)',
 'Draw circle (counter clockwise)']

In [13]:
from tqdm.autonotebook import tqdm

skeletons, target, video_ids = [],[],[]
for class_type in tqdm(set_of_classes):
  class_loc = os.path.join(data_fol,class_type)
  video_list = os.listdir(class_loc)

  if len(video_list)>300:
    video_list = video_list[:10]
  for vid in tqdm(video_list):
    video_id = vid.split(".")[0]

    if os.path.exists(f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/TEMP/{class_type}_{video_id}.npz"):
      continue

    skel_res, fh, fw = pose_estimator(pose, os.path.join(class_loc,vid))
    if skel_res.shape[0] != 0:

      np.savez(
      f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Devin/TEMP/{class_type}_{video_id}.npz",
      np.array(class_type),
      skel_res,
      np.array(video_id)
      )

      skeletons.append(skel_res)
      target.append(class_type)
      video_ids.append(video_id)




<ipython-input-13-dcb32c8955a1>:1: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


  0%|          | 0/32 [00:00<?, ?it/s]

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

  0%|          | 0/32 [00:00<?, ?it/s]

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

...

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

  0%|          | 0/32 [00:00<?, ?it/s]

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

  0%|          | 0/32 [00:00<?, ?it/s]

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [14]:
joints = [11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]

In [15]:
skeletons0 = [x[:,joints,:] for x in skeletons]

In [16]:
for i in range(len(skeletons0)):
  skel = skeletons0[i]

  if skel.shape[0] <65:
    times = 60//skel.shape[0] +1
    skel = np.tile(skel,(times,1,1))

  mid_point = skel.shape[0]//2
  skeletons0[i] = skel[mid_point-30:mid_point+30,...]

skeletons0 = np.stack(skeletons0,axis=0)
skeletons0 = skeletons0.reshape(skeletons0.shape[0],skeletons0.shape[1],skeletons0.shape[2]*skeletons0.shape[3])

In [17]:
skeletons0.shape

(861, 60, 36)

In [19]:
np.savez(
    "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/Consolidated/UTD-MHAD/Skeletons/skeleton_mediapipe_new.npz",
    np.array(target),
    skeletons0,
    np.array(video_ids)
    )

In [22]:
file_load = np.load("/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/Consolidated/UTD-MHAD/Skeletons/skeleton_mediapipe_new.npz")

In [23]:
file_load["arr_0"].shape,file_load["arr_1"].shape,file_load["arr_2"].shape

((861,), (861, 60, 36), (861,))

In [ ]:
def gen_skeleton(frame, 
                 height,
                 width,
                 mapping_list = [(0, 1), (1, 3), (3, 5), 
                                 (0, 2), (2, 4), (0, 6), 
                                 (1, 7), (6, 7), (6, 8), 
                                 (7, 9), (8, 10), (9, 11)]):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3

def gen_video(points, 
              save_file, 
              frame_h, 
              frame_w, 
              is_3d=True,
              mapping_list = [(0, 1), (1, 3), (3, 5), 
                                 (0, 2), (2, 4), (0, 6), 
                                 (1, 7), (6, 7), (6, 8), 
                                 (7, 9), (8, 10), (9, 11)]):
    # make 3D if points are flatten
    if len(points.shape) == 2:
        if is_3d:
          fts = points.shape[1]
          x_cds = list(range(0, fts, 3))
          y_cds = list(range(1, fts, 3))
          z_cds = list(range(2, fts, 3))
          points = np.transpose(np.array([points[:, x_cds], 
                                          points[:, y_cds], 
                                          points[:, z_cds]]), (1,2,0))
        else:
          fts = points.shape[1]
          x_cds = list(range(0, fts, 2))
          y_cds = list(range(1, fts, 2))
          points = np.transpose(np.array([points[:, x_cds], 
                                          points[:, y_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    for __id,frame in enumerate(points):
        skel_image = gen_skeleton(frame, frame_h, frame_w,mapping_list=mapping_list)
        result.write(skel_image)

    result.release()

In [ ]:
!mkdir gen_data

In [ ]:
for t,x,id in tqdm(zip(file_load["arr_0"],file_load["arr_1"],file_load["arr_2"])):
  gen_video(
      x,
      f"gen_data/{t}_{id}.mp4",
      400,
      400
  )

In [ ]:
np.unique(file_load["arr_0"])

In [ ]:
!zip gen_data

In [ ]:
"""data_fol = "/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/HAR_Video/K10_V2"
path_parts = data_fol.split("/Datasets/")[1]

model_ident = "MediaPose"
set_of_classes = [x for x in os.listdir(data_fol)]
main_npz_directory = f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_npz_outputs/{model_ident}/{path_parts}"
main_vid_directory = f"/content/drive/MyDrive/22_FYP42 - Zero-shot Explainable HAR/Datasets/skeleton_reconstruction_outputs/{model_ident}/{path_parts}"""

In [ ]:
"""for class_type in set_of_classes:
  class_loc = os.path.join(data_fol,class_type)
  save_fol = os.path.join(main_vid_directory,class_type)
  save_numpy_fol = os.path.join(main_npz_directory,class_type)
  os.makedirs(save_fol,exist_ok=True)
  os.makedirs(save_numpy_fol,exist_ok=True)
  video_list = os.listdir(class_loc)

  if len(video_list)>300:
    video_list = video_list[:300]
  for vid in tqdm(video_list):
    video_id = vid.split(".")[0]
    if not os.path.exists(os.path.join(save_fol,f"{video_id}.mp4")):
      skel_res, fh, fw = pose_estimator(pose, os.path.join(class_loc,vid))
      if skel_res.shape[0] != 0:
        save_arrays_file(skel_res,[fh,fw],os.path.join(save_numpy_fol,f"{video_id}.npz"))
        gen_video(skel_res, os.path.join(save_fol,f"{video_id}.mp4"),video_id, fh, fw)
    else:
      print(f'{os.path.join(save_fol,f"{video_id}.mp4")} path exists !')"""